In [1]:
from bs4 import BeautifulSoup
import requests
import csv

In [2]:
# Define keywords
key = 'university'
location = 'UnitedKingdom'

In [3]:
# Put keywords to URL
url = 'https://www.yell.com/ucs/UcsSearchAction.do?keywords={}&location={}&scrambleSeed=670705130&pageNum='.format(key,location)

In [4]:
# To by pass if response not 200, we need parameter
heads = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36 Edg/97.0.1072.62'
}

In [5]:
# Make data by looping every web pages
datas = []
for page in range(1, 5):
  req = requests.get(url+str(page), headers=heads)
  soup = BeautifulSoup(req.text, 'html.parser')
  items = soup.findAll('div', 'row businessCapsule--mainRow')
  for it in items:
    Name = ''.join(it.find('h2', 'businessCapsule--name text-h2').text.strip().split('\n'))

    Address = it.find('span', {'itemprop':'address'})
    if Address is not None :
      Address = ''.join(Address.text.strip().split('\n'))
    else:
      Address = None
  
    try : Website = it.find('a', {'rel':'nofollow noopener'})['href'].replace('http://www.', '').replace('https://www.', '').replace('http://WWW.', '').replace('https://WWW.', '').split('/')[0]
    except : Website = ''
  
    try : Telephone = it.find('span', 'business--telephoneNumber').text
    except : Telephone = ''

    Image = it.find('div', 'col-sm-4 col-md-4 col-lg-5 businessCapsule--leftSide').find('img')['data-original']
    if 'http' not in Image: Image = 'https://www.yell.com{}'.format(Image)
    
    datas.append([Name, Address, Website, Telephone, Image])

In [6]:
# Make columns name
table_headers = ['Name', 'Address', 'Website', 'Phone', 'Image URL']

# write parameter
writer = csv.writer(open('sample_data/{}_{}.csv'.format(key,location), 'w', newline=''))

# fill row with page content
writer.writerow(table_headers)
for d in datas: writer.writerow(d)

In [8]:
# Check dataframe
import pandas as pd
df = pd.read_csv('sample_data/university_UnitedKingdom.csv')
df.sample(10)

,Name,Address,Website,Phone,Image URL
43,Jason Thomas Performing Arts,NaN,NaN,01872 276365,"https://c.yell.com/t_bigRect,f_auto/c6881d41-3..."
71,Anne Williams-King Singing Teacher,"Groes Stryt Farm Pen Y Bryn, Pen-y-Cae, Wrexha...",NaN,07973 135869,https://www.yell.com/searchfe/yframe/yelldotco...
38,South Harrow Tutorials,"112 Torbay Rd, Harrow, HA2 9QJ",NaN,07912 353915,"https://c.yell.com/t_bigSquare,f_auto/bd6aedfc..."
13,Jamie's Tuition,NaN,jamiestuition.co.uk,07883 064185,"https://c.yell.com/t_bigRect,f_auto/35020af8-d..."
50,Bex Harper Earth Coaching,NaN,bexharper.com?utm_source=yell&utm_medium=refer...,07526 938805,"https://c.yell.com/t_bigRect,f_auto/f564c2e0-8..."
73,Hooton Tutors,NaN,NaN,0151 327 2155,"https://c.yell.com/t_bigRect,f_auto/e036aa26-a..."
3,Smart Move Education Group Ltd,"Nile Business Center 56-60, Nelson St, London,...",NaN,020 7790 9233,"https://c.yell.com/t_bigRect,f_auto/930374cd-8..."
4,University Of Northampton,"Waterside University Drive, Northampton, NN1 5PH",northampton.ac.uk,01604 735500,"https://c.yell.com/t_bigSquare,f_auto/b1564945..."
85,Rush Resources Tuition Centre,"39 Glebelands Avenue, Ilford, IG2 7DN",rushresources.co.uk,07827 277948,"https://c.yell.com/t_bigRect,f_auto/2177e472-5..."
54,Conversation to Degree,"2 Gellyfowy Rd Ynysmeudwy, Swansea, SA8 4QE",NaN,07813 834566,"https://c.yell.com/t_bigRect,f_auto/dd39b259-8..."
